# Import Data

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
import os.path
import math
from xgboost import XGBClassifier
from sklearn.externals import joblib
import pandas as pd

n_bytes = 2**31
max_bytes = 2**31 - 1

class Time_Tracking():
    
    start_time = None
    
    def start_tracking(self):
        
        self.start_time = time.time()
    
    def stop_tracking(self):
        
        print("Time used:", round(((time.time() - self.start_time)/60),2), ' minutes')


def load_pickle(file_path):
    
    bytes_in = bytearray(0)
    input_size = os.path.getsize(file_path)
    with open(file_path, 'rb') as f_in:
        for _ in range(0, input_size, max_bytes):
            bytes_in += f_in.read(max_bytes)
    
    return pickle.loads(bytes_in)


In [2]:
model = joblib.load("xgb.pickle.dat")

train = load_pickle('train_df_after_preprocessing.pkl')
valid = load_pickle('valid_df_after_preprocessing.pkl')
test = load_pickle('test_df_after_preprocessing.pkl')

valid_click = valid['click']
valid_bidprice = valid['bidprice']
valid_payprice = valid['payprice']
valid_X = valid.drop(['click', 'bidprice', 'payprice'], axis = 1)

### 暂时没时间 tune xgboost, 先 tune 各种strategy

In [3]:
# strategy 1: linear bidding(base_line)
# strategy 2: OCTR
# strategy 3: threshold linear bidding 
# strategy 4: linear bidding with floor price 
# strategy 5: sigmoid threshold bid 

# threshold 可能效果不是很好，因为 threshold = 0.5 的时候，failed = 61, 

In [3]:
valid_ctr_prediction = model.predict_proba(valid_X)

In [4]:
pCTR = valid_ctr_prediction[:,1]
bid_price = [1644*p for p in pCTR]
auction = bid_price >= valid_payprice
win_bid_ix = [index for index,auc in enumerate(auction) if auc]

clicks = sum(valid_click[win_bid_ix])
spend = sum(valid_payprice[win_bid_ix])
num_impress = len(win_bid_ix)

sorted_pCTR_index = sorted(range(len(auction)), key=lambda k:pCTR[k])

In [10]:
np.arange(,0,-1)

array([5, 4, 3, 2, 1])

In [41]:
auction = bid_price >= valid_payprice
win_bid_ix = [ index for index,auc in enumerate(auction) if auc ]
bid_spends = valid_payprice[win_bid_ix].values
bid_clicks = valid_click[win_bid_ix].values

budget = 6250000
current_spend = sum(bid_spends)

total_bids = len(bid_spends)

for i in range(len(bid_spends)):
    
    current_spend -= bid_spends[total_bids-i-1]
    
    if current_spend <= budget:
        break

final_win_bid_ix = win_bid_ix[:(total_bids - i -1)]
bid_spends = sum(valid_payprice[final_win_bid_ix].values)
bid_clicks = sum(valid_click[final_win_bid_ix].values)

In [12]:
sum(valid_payprice[win_bid_ix[:302377]]) <= budget

False

In [44]:
def EvaluateClick( bid_price, budget, valid_payprice, valid_click):
    
    auction = bid_price >= valid_payprice
    win_bid_ix = [ index for index,auc in enumerate(auction) if auc ]
    bid_spends = valid_payprice[win_bid_ix].values
    bid_clicks = valid_click[win_bid_ix].values

    budget = 6250000
    current_spend = sum(bid_spends)

    total_bids = len(bid_spends)

    for i in range(len(bid_spends)):

        current_spend -= bid_spends[total_bids-i-1]

        if current_spend <= budget:
            break

    final_win_bid_ix = win_bid_ix[:(total_bids - i -1)]
    bid_spends = sum(valid_payprice[final_win_bid_ix].values)
    bid_clicks = sum(valid_click[final_win_bid_ix].values)
    
    print('clicks:', bid_clicks, 'impressions:', len(final_win_bid_ix), 'spend:', bid_spends)

    return bid_clicks


# bid_price = 1644*valid_ctr_prediction[:,1]
# EvaluateClick( bid_price, 6250000, valid_payprice, valid_click);

clicks: 51 impressions: 80425 spend: 6249997


In [57]:
# # search the best base_price

# base_prices = np.arange(200,250,1)
# clicks = []
# budget = 6250000

# for base in base_prices:
    
#     pCTR = valid_ctr_prediction[:,1]
#     bid_price = base*pCTR
#     c = EvaluateClick( bid_price, budget, valid_payprice, valid_click);
#     clicks.append(c)

    
# plt.figure()
# plt.plot(base_prices, clicks)
# print('best constant:', base_prices[clicks.index(max(clicks))])

### The best constant is 214. With 145 clicks on validation set. 

In [122]:
### prediction
test_prediction = model.predict_proba(test)[:,1]
bid_price = [int(214*t) for t in test_prediction]

submission = pd.read_csv('Group_xx.csv')
submission['bidprice'] = bid_price
submission.to_csv('Group_9.csv')

In [119]:
max(submission['bidprice'])

213

# 通过 api 测试，得到 click = 162.

# Threshold sigmoid bidding: 1/( 1+exp^(-z) )

In [183]:
# sigmoid = [ 1/( 1 + math.exp(-(z-0.9))) for z in valid_ctr_prediction[:,1] ]

# base_prices =  np.arange(188,200,1)
# spend = []
# clicks = []

# for base in base_prices:
    
#     bid_price = [base*s for s in sigmoid]
#     c = EvaluateClick( bid_price, budget , valid_payprice, valid_click)
#     clicks.append(c)

# plt.figure()
# plt.plot(base_prices, clicks)
# print('best constant:', base_prices[clicks.index(max(clicks))])

### The best constant is 181. With 108 clicks on the validation set. Constraint on (z-0.8)

In [111]:
submission = pd.read_csv('Group_xx.csv')
test_prediction = model.predict_proba(test)[:,1]
sigmoid = [ 1/( 1 + math.exp(-(z-0.9))) for z in test_prediction ]
bid_price = [int(181*s) for s in sigmoid]

submission['bidprice'] = bid_price
submission.to_csv('Group_9.csv')

# ORTB: bidding_price = sqrt( pCTR*C\lambda + C^2) - C

In [175]:
valid_pctr = valid_ctr_prediction[:,1] 
lamda = 5.2*10**(-4)
c = 10.41

bid_price = [ int(math.sqrt(pCTR * c / lamda + c**2) - c ) for pCTR in valid_pctr ]
EvaluateClick( bid_price, budget , valid_payprice, valid_click)

clicks: 124 impressions: 158508 spend: 6159692


124

In [176]:
# C = np.arange(10.4,10.6,0.01)
# clicks = []

# for c in C:
    
#     bid_price = [ int(math.sqrt(pCTR * c / lamda + c**2) - c ) for pCTR in valid_pctr ]
#     click = EvaluateClick( bid_price, budget , valid_payprice, valid_click)
#     clicks.append(click)

# plt.figure()
# plt.plot(C, clicks)
# print('best constant:', C[clicks.index(max(clicks))])

### The best click is 124, with lambda = 5.2 * 10**(-4), c = 10.41

In [179]:
submission = pd.read_csv('Group_xx.csv')
test_prediction = model.predict_proba(test)[:,1]
lamda = 5.2*10**(-4)
c = 10.41
bid_price = [ int(math.sqrt(pCTR * c / lamda + c**2) - c ) for pCTR in test_prediction ]

submission['bidprice'] = bid_price
submission.to_csv('Group_9.csv')

# 通过 api 测试，得到 click = 125. 可能还需要继续 tune lamda 和 c